In [1]:
import pandas as pd
import seaborn as sns

from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer  

import re

from sklearn.naive_bayes import MultinomialNB

import numpy as np
import matplotlib.pyplot as plt
from gensim.test.utils import datapath, get_tmpfile
from gensim.models import KeyedVectors
from gensim.scripts.glove2word2vec import glove2word2vec

from sklearn.model_selection import train_test_split
from sklearn.neural_network import MLPClassifier
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.pipeline import make_pipeline

import seaborn as sns; sns.set()
from sklearn.metrics import confusion_matrix

from sklearn.metrics import f1_score
from sklearn.metrics import accuracy_score

import nltk
nltk.download('stopwords')
import string

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [2]:
from google.colab import drive
drive.mount('/content/drive')
data = pd.read_csv('/content/drive/MyDrive/nlp project/data.csv')
data

Mounted at /content/drive


,Unnamed: 0,targetCategory,post,y
0,0,race,RT @iBeZo: Stupid fucking nigger LeBron. You f...,1
1,1,race,I need a girl from Jamaica I can't fuck with t...,1
2,2,gender,Molly make bitches crazy,2
3,3,race,This niggga said I be branding bitches &#12856...,1
4,4,race,It's funny when u get a black female mad they ...,1
...,...,...,...,...
12075,12075,gender,Does anyone else have the suspicion that moder...,2
12076,12076,disabled,We need World War III\nThen all Chads and Norm...,5
12077,12077,gender,Roasties can look like this and STILL be consi...,2
12078,12078,social,Help me find a video\nI'm looking for a video ...,6


In [3]:
X = data.iloc[:, [2,1]]
X

,post,targetCategory
0,RT @iBeZo: Stupid fucking nigger LeBron. You f...,race
1,I need a girl from Jamaica I can't fuck with t...,race
2,Molly make bitches crazy,gender
3,This niggga said I be branding bitches &#12856...,race
4,It's funny when u get a black female mad they ...,race
...,...,...
12075,Does anyone else have the suspicion that moder...,gender
12076,We need World War III\nThen all Chads and Norm...,disabled
12077,Roasties can look like this and STILL be consi...,gender
12078,Help me find a video\nI'm looking for a video ...,social


In [4]:
sw = stopwords.words('english')
lemmatizer = WordNetLemmatizer()



def cleantext(text):
    text= text.lower()
    text= re.sub(r"[^a-zA-Z?.!,¿]+", " ", text)
    text= re.sub(r"http\S+", "",text)
    text= re.sub(r"http", "",text)
    
    punctuations= '@#!?+&*[]-%.:/();$=><|{}^' + "'`" + '_'
    for p in punctuations:
        text= text.replace(p, '')
        
    text= [word.lower() for word in text.split() if word.lower() not in sw]
    
    text= [lemmatizer.lemmatize(word) for word in text]
    
    text = " ".join(text)
    
    emoji_pattern = re.compile("["
                           u"\U0001F600-\U0001F64F"  
                           u"\U0001F300-\U0001F5FF"  
                           u"\U0001F680-\U0001F6FF"  
                           u"\U0001F1E0-\U0001F1FF"  
                           u"\U00002702-\U000027B0"
                           u"\U000024C2-\U0001F251"
                           "]+", flags=re.UNICODE)
    
    text= emoji_pattern.sub(r'',text)
    
    return text

In [5]:
nltk.download('wordnet')
nltk.download('omw-1.4')

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...


True

In [6]:
X['post'] = X['post'].apply(lambda x: cleantext(x))
 
X.head() 

<ipython-input-6-b1fd811f9254>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X['post'] = X['post'].apply(lambda x: cleantext(x))


,post,targetCategory
0,rt ibezo stupid fucking nigger lebron flopping...,race
1,need girl jamaica fuck basic black bitch,race
2,molly make bitch crazy,gender
3,niggga said branding bitch,race
4,funny u get black female mad start acting ghet...,race


In [7]:
from sklearn.preprocessing import LabelEncoder# Initialize the Label Encoder.
le = LabelEncoder()

# Encode the categories
X['TargetCategory_enc'] = le.fit_transform(data['targetCategory'])

# Display the first five rows again to see the result
display(X.head())

# Print the datatypes
print(X.dtypes)

<ipython-input-7-6a7a233d6453>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X['TargetCategory_enc'] = le.fit_transform(data['targetCategory'])


,post,targetCategory,TargetCategory_enc
0,rt ibezo stupid fucking nigger lebron flopping...,race,4
1,need girl jamaica fuck basic black bitch,race,4
2,molly make bitch crazy,gender,3
3,niggga said branding bitch,race,4
4,funny u get black female mad start acting ghet...,race,4


post                  object
targetCategory        object
TargetCategory_enc     int64
dtype: object


In [8]:
from sklearn.model_selection import train_test_split

A = X.post
y = X.targetCategory

X_train, X_test, y_train, y_test = train_test_split(A, y, test_size=0.2, random_state = 42)

## tfidf + naive bayesian

In [9]:
m = make_pipeline(TfidfVectorizer(), MultinomialNB())

In [10]:
m.fit(X_train, y_train)

Pipeline(steps=[('tfidfvectorizer', TfidfVectorizer()),
                ('multinomialnb', MultinomialNB())])

In [11]:
y_pred_em = m.predict(X_test)
y_pred_em

array(['race', 'gender', 'race', ..., 'gender', 'culture', 'race'],
      dtype='<U8')

In [12]:
confusionmat_embeded = confusion_matrix(y_test, y_pred_em)
confusionmat_embeded

array([[  0,   3,   0,  43,  27,   0,   6],
       [  0, 227,   0,  44, 146,   0,   6],
       [  0,   7,  12,  57,  82,   0,   1],
       [  0,   1,   0, 557,  62,   0,   2],
       [  0,  18,   0,  51, 631,   0,   4],
       [  0,  10,   0,  53,  52,   1,   1],
       [  0,  28,   0, 113, 113,   0,  58]])

In [13]:
# 80/20
accuracy = accuracy_score(y_test, y_pred_em)
microaveraged_F1_score = f1_score(y_test, y_pred_em, average='micro')
macroaveraged_F1_score = f1_score(y_test, y_pred_em, average='macro')
print("Architecture 1 accuracy:", accuracy)
print("Architecture 1 microaveraged F1 score:", microaveraged_F1_score)
print("Architecture 1 macroaveraged F1 score:", macroaveraged_F1_score)

Architecture 1 accuracy: 0.6150662251655629
Architecture 1 microaveraged F1 score: 0.6150662251655629
Architecture 1 macroaveraged F1 score: 0.3579796960450362


# TF-IDF + adaboost

In [14]:
from sklearn.ensemble import AdaBoostClassifier

In [15]:
m = make_pipeline(TfidfVectorizer(), AdaBoostClassifier())
m.fit(X_train, y_train)

Pipeline(steps=[('tfidfvectorizer', TfidfVectorizer()),
                ('adaboostclassifier', AdaBoostClassifier())])

In [16]:
y_pred_em = m.predict(X_test)
y_pred_em

array(['social', 'victim', 'disabled', ..., 'gender', 'victim', 'race'],
      dtype=object)

In [17]:
confusionmat_embeded = confusion_matrix(y_test, y_pred_em)
confusionmat_embeded

array([[ 18,   0,   1,  12,   2,   1,  45],
       [  0, 210,   5,  16,  13,   6, 173],
       [  1,   0,  87,   5,   5,   1,  60],
       [  3,   1,   3, 424,  35,  10, 146],
       [  2,  11,   6,  23, 485,  13, 164],
       [  0,   2,   1,   7,   7,  54,  46],
       [  3,  17,   6,  30,   6,   2, 248]])

In [18]:
accuracy = accuracy_score(y_test, y_pred_em)
microaveraged_F1_score = f1_score(y_test, y_pred_em, average='micro')
macroaveraged_F1_score = f1_score(y_test, y_pred_em, average='macro')
print("Architecture 1 accuracy:", accuracy)
print("Architecture 1 microaveraged F1 score:", microaveraged_F1_score)
print("Architecture 1 macroaveraged F1 score:", macroaveraged_F1_score)

Architecture 1 accuracy: 0.6316225165562914
Architecture 1 microaveraged F1 score: 0.6316225165562914
Architecture 1 macroaveraged F1 score: 0.5832028533723388


# TF-IDF + MLP

In [19]:
m = make_pipeline(TfidfVectorizer(), MLPClassifier(hidden_layer_sizes=(200,100,20,),verbose=True))
m.fit(X_train, y_train)

Iteration 1, loss = 1.68145753
Iteration 2, loss = 1.03810643
Iteration 3, loss = 0.52309064
Iteration 4, loss = 0.28829654
Iteration 5, loss = 0.15780602
Iteration 6, loss = 0.09370696
Iteration 7, loss = 0.05814675
Iteration 8, loss = 0.04278263
Iteration 9, loss = 0.03302918
Iteration 10, loss = 0.02771749
Iteration 11, loss = 0.02411537
Iteration 12, loss = 0.02032889
Iteration 13, loss = 0.01767854
Iteration 14, loss = 0.01903520
Iteration 15, loss = 0.01775122
Iteration 16, loss = 0.01495081
Iteration 17, loss = 0.01673713
Iteration 18, loss = 0.01732497
Iteration 19, loss = 0.01442638
Iteration 20, loss = 0.01230293
Iteration 21, loss = 0.01426870
Iteration 22, loss = 0.01170825
Iteration 23, loss = 0.01241689
Iteration 24, loss = 0.01335733
Iteration 25, loss = 0.01286736
Iteration 26, loss = 0.01244438
Iteration 27, loss = 0.01251862
Iteration 28, loss = 0.01112557
Iteration 29, loss = 0.01087527
Iteration 30, loss = 0.01007883
Iteration 31, loss = 0.01094620
Iteration 32, los

Pipeline(steps=[('tfidfvectorizer', TfidfVectorizer()),
                ('mlpclassifier',
                 MLPClassifier(hidden_layer_sizes=(200, 100, 20),
                               verbose=True))])

In [20]:
y_pred_em = m.predict(X_test)
y_pred_em

array(['social', 'victim', 'disabled', ..., 'gender', 'culture', 'race'],
      dtype='<U8')

In [21]:
accuracy = accuracy_score(y_test, y_pred_em)
microaveraged_F1_score = f1_score(y_test, y_pred_em, average='micro')
macroaveraged_F1_score = f1_score(y_test, y_pred_em, average='macro')
print("Architecture 2 accuracy:", accuracy)
print("Architecture 2 microaveraged F1 score:", microaveraged_F1_score)
print("Architecture 2 macroaveraged F1 score:", macroaveraged_F1_score)

Architecture 2 accuracy: 0.6564569536423841
Architecture 2 microaveraged F1 score: 0.6564569536423841
Architecture 2 macroaveraged F1 score: 0.5491219697955746


## golve + rnn

In [22]:
import torch
import torch.nn as nn
from torch.nn.utils.rnn import pack_padded_sequence, pad_packed_sequence
from torch.utils.data import Dataset, DataLoader
from torch import cuda
device = 'cuda' if cuda.is_available() else 'cpu'

In [23]:
A = X.post
y = X.TargetCategory_enc

X_train, X_test, y_train, y_test = train_test_split(A, y, test_size=0.2, random_state = 42)

In [24]:
sw_set=stopwords.words('english')+list(string.punctuation)

In [25]:
glove_file = datapath('/content/drive/MyDrive/nlp project/glove.6B.100d.txt')
word2vec_text_file = get_tmpfile("glove.6B.100d.word2vec.txt")
glove2word2vec(glove_file, word2vec_text_file)

(400001, 100)

In [26]:
model_em = KeyedVectors.load_word2vec_format(word2vec_text_file)

In [27]:
## use my impelementation of calc_doc_embedding
OOV_embed = dict()
def calc_doc_embedding(doc, model, embed_size, OOV_embed):
    
    words = doc.split(" ")
    doc_embed = np.zeros((100,100))
    for i in range(len(words)):
        word = words[i]
        if word not in stopset:
            try: 
                embed = model.get_vector(word)
            except:
                try:
                    embed = OOV_embed[word]
                except:
                    embed = np.random.rand(embed_size)
                    OOV_embed[word] = embed
                    
            doc_embed[i] = embed

    return doc_embed

stopset = stopwords.words('english') + list(string.punctuation)

In [28]:
def calc_doc_length(doc, model, embed_size, OOV_embed):
    
    words = doc.split(" ")
    doc_len = 0
    for i in range(len(words)):
        word = words[i]
        if word not in stopset:
            try: 
                embed = model.get_vector(word)
            except:
                try:
                    embed = OOV_embed[word]
                except:
                    embed = np.random.rand(embed_size)
                    OOV_embed[word] = embed
                    
            doc_len += 1

    return doc_len

In [29]:
X_train_len = np.asarray([calc_doc_length(d, model_em, 100, OOV_embed) for d in X_train])
X_test_len = np.asarray([calc_doc_length(d, model_em, 100, OOV_embed) for d in X_test])

In [30]:
X_train_embeded = np.array([calc_doc_embedding(d, model_em, 100, OOV_embed) for d in X_train])
X_test_embeded = np.array([calc_doc_embedding(d, model_em, 100, OOV_embed) for d in X_test])

In [31]:
class CustomDataset(Dataset):
    def __init__(self, X, Y, x_length):
        self.X = X
        self.y = Y
        self.x_length = x_length
        
    def __len__(self):
        return len(self.y)
    
    def __getitem__(self, idx):
        return  torch.from_numpy(self.X[idx].astype(np.float64)), torch.from_numpy(self.y[idx].astype(np.int32)),self.x_length[idx]

In [32]:
def to_one_hot(labels, dimension=7):
  results = np.zeros(dimension)
  results[labels-1] = 1
  return results.astype(int)

y_train_embeded = y_train.apply(to_one_hot).values
y_test_embeded = y_test.apply(to_one_hot).values

In [33]:
training_set = CustomDataset(X_train_embeded, y_train_embeded, X_train_len)
testing_set = CustomDataset(X_test_embeded, y_test_embeded, X_test_len)

In [34]:
batch_size = 256
training_loader = DataLoader(training_set, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(testing_set, batch_size=batch_size)

In [35]:
for X,y,l in training_loader:
  a = X

In [36]:
class LSTM_variable_input(torch.nn.Module) :
    def __init__(self, embedding_dim, hidden_dim) :
        super().__init__()
        self.hidden_dim = hidden_dim
        self.lstm = nn.LSTM(embedding_dim, hidden_dim, batch_first=True)
        self.linear = nn.Linear(hidden_dim, 7)
        
    def forward(self, x, x_length):
        x_pack = pack_padded_sequence(x, x_length, batch_first=True, enforce_sorted=False)
        out_pack, (ht, ct) = self.lstm(x_pack)
        out = self.linear(ht[-1])
        return out

In [37]:
model = LSTM_variable_input(100, 100)
model.to(device)

LSTM_variable_input(
  (lstm): LSTM(100, 100, batch_first=True)
  (linear): Linear(in_features=100, out_features=7, bias=True)
)

In [38]:
def BCE_loss(outputs, targets):
    return torch.nn.BCEWithLogitsLoss()(outputs, targets)

In [39]:
optimizer = torch.optim.Adam(params =  model.parameters(), lr=0.0001)
loss_fn = BCE_loss

In [46]:
def train(epoch):
    model.train()
    for X,y,l in training_loader:

        outputs = model(X.float().to(device),l)

        optimizer.zero_grad()

        loss = loss_fn(outputs.to(device), y.float().to(device))

        print(f'Epoch: {epoch}, Loss:  {loss.item()}')
        
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

In [47]:
EPOCHS = 100
for epoch in range(EPOCHS):
  print(epoch)
  train(epoch)

0
Epoch: 0, Loss:  0.6918531656265259
Epoch: 0, Loss:  0.6892716288566589
Epoch: 0, Loss:  0.6896129846572876
Epoch: 0, Loss:  0.6883929967880249
Epoch: 0, Loss:  0.6870137453079224
Epoch: 0, Loss:  0.6846478581428528
Epoch: 0, Loss:  0.6845117211341858
Epoch: 0, Loss:  0.6839481592178345
Epoch: 0, Loss:  0.683352530002594
Epoch: 0, Loss:  0.6815875768661499
Epoch: 0, Loss:  0.6798282265663147
Epoch: 0, Loss:  0.6794918775558472
Epoch: 0, Loss:  0.6781125664710999
Epoch: 0, Loss:  0.6774775385856628
Epoch: 0, Loss:  0.6753278970718384
Epoch: 0, Loss:  0.6720530986785889
Epoch: 0, Loss:  0.6730408072471619
Epoch: 0, Loss:  0.6713625192642212
Epoch: 0, Loss:  0.6715277433395386
Epoch: 0, Loss:  0.6714311838150024
Epoch: 0, Loss:  0.6671625375747681
Epoch: 0, Loss:  0.6673532128334045
Epoch: 0, Loss:  0.6663879752159119
Epoch: 0, Loss:  0.6659590005874634
Epoch: 0, Loss:  0.6648502349853516
Epoch: 0, Loss:  0.6647365689277649
Epoch: 0, Loss:  0.6613242626190186
Epoch: 0, Loss:  0.66067498

In [50]:
def validation():
    model.eval()
    with torch.no_grad():
        for X,y,l in test_loader:
            outputs = model(X.float().to(device),l)
    return outputs, y

In [51]:
outputs, y_test = validation()

In [52]:
def output2predict(outputs, dim = 7):
  c = np.argmax(outputs)
  p = np.zeros(dim).astype(int)
  p[c] = 1
  return p

In [54]:
y_pred_em = np.array([output2predict(output.cpu()) for output in outputs])

In [55]:
accuracy = accuracy_score(y_test, y_pred_em)
microaveraged_F1_score = f1_score(y_test, y_pred_em, average='micro')
macroaveraged_F1_score = f1_score(y_test, y_pred_em, average='macro')
print("Architecture 2 accuracy:", accuracy)
print("Architecture 2 microaveraged F1 score:", microaveraged_F1_score)
print("Architecture 2 macroaveraged F1 score:", macroaveraged_F1_score)

Architecture 2 accuracy: 0.7053571428571429
Architecture 2 microaveraged F1 score: 0.7053571428571429
Architecture 2 macroaveraged F1 score: 0.5216734073177507
